In [1]:
import torch
import numpy as np
import pandas as pd
import json
from ipywidgets import interact
from joblib import load
from modules.cGAN_samplers import noise_sampler
from modules.functions import decode, calculate_entropy_mixing

In [8]:
model = torch.jit.load('saved_cGAN/generator.pt',map_location='cpu')
with open('saved_cGAN/gan_hyperparameters.json','r') as fid:
    gan_hyperparameters = json.load(fid)
selected_props = gan_hyperparameters['selected_props']
latent_dim = gan_hyperparameters['latent_dim']
prop_dim = gan_hyperparameters['prop_dim']
elem_list = gan_hyperparameters['elem_list']

scaler_pipe = load('saved_cGAN/scale_kde_pipe.joblib')

In [9]:
@interact(DELTA_S =(-2, -0.2,0.01), D_PARAM=(1, 4,0.01), PRICE = (10, 800),FT = (4,22,0.01),DENSITY=(5,20,0.01),HARDNESS=(0.1,10,0.01),UTS1200C = (0.1,2,0.01))
def predict_house_price(DELTA_S,D_PARAM,PRICE,FT,DENSITY,HARDNESS,UTS1200C):
    input_data = np.array([[DELTA_S,D_PARAM,PRICE,FT,DENSITY,HARDNESS,UTS1200C]]).reshape(1,-1).astype('float32')
    scaled_input = torch.from_numpy(scaler_pipe['Scaler'].transform(input_data))
    noise = torch.from_numpy(noise_sampler(1,latent_dim))
    with torch.no_grad():
        generated = model(noise,scaled_input)
    composition = decode(generated[0], elem_list)
    res = np.zeros((1,len(selected_props)))
    for i,prop in enumerate(selected_props):
        if prop == 'delta_S':
            res[:,0] = calculate_entropy_mixing(composition)
        else:
            model_surrogate = load(f'saved_surrogates/surrogate_{prop}.joblib')
            res[:,i] = model_surrogate.predict(generated.numpy())

    print(f'Generated composition with given conditions : {composition}')
    print(pd.DataFrame(res, columns=selected_props))


interactive(children=(FloatSlider(value=-1.1099999999999999, description='DELTA_S', max=-0.2, min=-2.0, step=0…